# Исследование надёжности заёмщиков

Для кредитного отдела банка на основе статистика о платёжеспособности клиентов необходимо выяснить, как влияют на факт погашения кредита в срок различные данные клиента.

## Общая информация

In [1]:
import math
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим информацию подробнее. В таблице имеется 12 столбцов трёх типов: вещественный, целочисленный и объектный.

Таблица состоит из следующих столбцов:

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

В столбцах days_employed и total_income явно есть пропущенные значения, судя по сводной информации.

In [2]:
data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Типы данных столбцов в целом соответсвуют характеру содержащихся данных, кроме столбца days_employed (общий трудовой стаж в днях), для которого выбран тип вещественный, вместо более очевидного целочисленного.

**Вывод**

Каждая строка таблицы содержит сведения о клиенте, который воспользовался кредитом, цель кредита и признак имевшихся задолженностей по возврату кредита. Для нашего исследования особенно важны столбцы children, family_status, debt. 

Необходимо решить несколько проблем с данными: 
    пропуски и нулевые значения в столбцах days_employed, total_income, dob_years, 
    преобразование в целочисленный тип столбца days_employed,
    отрицательные значения в столбцах children и days_employed,
    нормализация регистра строковых значений в полях education, family_status.
    

## Предобработка данных

### Обработка пропусков

In [3]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
data.loc[data['days_employed'].isnull() == True].tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


Данные с пропущенными значениями в столбцах days_employed и total_income находятся в одних и тех же строках. Наличие пропусков не зависит явно от данных в других столбцах. Имеется порядка 10% таких строк от общего количества, строки содержат полезную информацию для анализа в важных для нас столбцах. Пропущенные значения имеют количественный характер, заменим их средними значениями.

Пропущенные значения для total_income заменим на среднее значение по столбцу.

In [5]:
total_income_avg = data['total_income'].mean() # среднее значение ежемесячного дохода

In [6]:
data['total_income'] = data['total_income'].fillna(total_income_avg)

Пропуски в столбце ежемесячного дохода заполним средним значением. Таким образом, мы не изменим среднее значение по столбцу и у нас не будет нежелательных всплесков в разбросе данных.

В поле days_employed имеются отрицательные и положительные значения.

In [7]:
data.loc[(data['days_employed'] > 0)]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [8]:
data.loc[(data['income_type'] == 'пенсионер')]['days_employed'].head()

4     340266.072047
12              NaN
18    400281.136913
24    338551.952911
25    363548.489348
Name: days_employed, dtype: float64

Прослеживается связь между аномально большими значениями в столбце стажа (~900/1000/1100 лет) и типом занятости "пенсионер". Вероятно, для пенсионеров в стаж подставлялись заведомо большие числа, чтобы ограничить анкеты от некоторых проверок. 
Отрицательные значения в поле days_employed вероятно образовались вычитанием даты на момент заполнения анкеты из указанной даты начала работы.

In [9]:
data['days_employed'].isnull().sum()

2174

В данных столбца days_employed есть большое число пропущенных данных. Поскольку стаж в основном зависит от возраста, то пустые значения возможно попробовать заполнить, сопоставив им средние значения стажа для каждого возраста. Построим таблицу со средним стажем для возрастов на имеющихся данных.

In [10]:
days_employed_data = data.loc[(data['days_employed'] < 0) & (data['dob_years'] != 0), ['dob_years', 'days_employed']]
average_days_employed = days_employed_data.groupby('dob_years').mean()

# средний стаж в днях в зависимости от возраста
average_days_employed.head()

,days_employed
dob_years,
19,-633.678086
20,-684.944308
21,-709.440930
22,-781.376775
23,-827.309437


Напишем функцию для заполнения недостающих значений, которые занесём в новый столбец. Эти данные можно будет использовать для разделения клиентов на категории по диапазонам стажа.

In [11]:
# функция устанавливает стаж по возрасту из average_days_employed
def update_days_employed_by_age(row):
    dob_years = row['dob_years']
    days_employed = row['days_employed']
    # если стаж не указан
    if math.isnan(days_employed):
        # выбираем средний стаж по возрасту
        if dob_years > 0:
            return abs(average_days_employed.loc[dob_years, 'days_employed'])
        else:
            return 0
    else:
        # оставляем как есть
        return abs(days_employed)
    
# создадим временный столбец
data['days_employed_final'] = data.apply(update_days_employed_by_age, axis=1)
data['days_employed'] = data['days_employed_final']
# удалим временный столбец
data = data.drop('days_employed_final', 1)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [12]:
data[data['dob_years'] == 0].count()

children            101
days_employed       101
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income        101
purpose             101
dtype: int64

Остались строки с нулевым возрастом, таких строк менее 1%. Т.к. столбец dob_years не влияет на наше исследование, удалим эти строки. Т. о.  мы сможем смело категоризовать таблицу по возрастным группам.

In [13]:
data = data.drop(data[(data['dob_years']==0)].index)

In [14]:
data['children'].value_counts()

 0     14080
 1      4802
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обнаружены аномальные значения количества детей: 20, -1. Невозможно вписать в имеющуюся прогрессию количество разновозрастных клиентов с 20 детьми. Эти данные скорее похожи на опечатки. Этот столбец не зависит явно от других столбцов. Так как процент строк с этими значениями менее 1% (122 строки), удалим их.

In [15]:
data.loc[(data['children']==20)].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
3302,20,2108.881612,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,167422.302208,профильное образование


In [16]:
# удаление записей, где аномальное количество детей
data = data.drop(data[(data['children']==20) | (data['children']==-1)].index)

Как уже упоминалось ранее, строк с количеством детей 20 и -1 в таблице ― 122. Это менее 1% от общего количества, что не вляется существенным для нашего анализа.

Убеждаемся, что удаление выполнено успешно.

In [17]:
data['children'].value_counts()

0    14080
1     4802
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

**Вывод**

In [18]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


 На этапе обработки пропусков были решены следующие проблемы:
    пропуски в столбцах days_employed, total_income, dob_years, 
    отрицательные значения в столбцах children и days_employed,
    удалены аномальные значения в столбце children.


### Замена типа данных

Приведём столбцы days_employed и total_income к целочисленному типу. Так как тип столбцов вещественный, то безопасно преобразовать дни методом округления до целого значения.

In [19]:
data = data.round({'days_employed': 0, 'total_income': 0}).astype({'days_employed': 'int', 'total_income': 'int'})
data.loc[:,['days_employed', 'total_income']].head()

,days_employed,total_income
0,8438,253876
1,4025,112080
2,5623,145886
3,4125,267629
4,340266,158616


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 12 columns):
children            21302 non-null int64
days_employed       21302 non-null int64
dob_years           21302 non-null int64
education           21302 non-null object
education_id        21302 non-null int64
family_status       21302 non-null object
family_status_id    21302 non-null int64
gender              21302 non-null object
income_type         21302 non-null object
debt                21302 non-null int64
total_income        21302 non-null int64
purpose             21302 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

Столбцы "общий трудовой стаж в днях" и "ежемесячный доход" переведены в целочисленный тип методом округления до целого. Столбец стажа вероятнее всего будет рассматриваться рамках попадания в годовые диапазоны, а доход - в рамках попадания в диапазоны десятков тысяч, выбранная точность округления более чем достаточна для такого разбиения данных.

### Обработка дубликатов

Проверим наличие полных повторений строк.

In [21]:
data.duplicated().sum()

54

Хотя наличие идентификатора клиента было бы большей гарантией уникальности, тем не менее, удалим полные дубликаты строк, т. к. предполагаем, что крайне маловероятно полное совпадение строк в такой широкой таблице с разноплановой информацией. 

In [22]:
data = data.drop_duplicates().reset_index(drop = True)

In [23]:
data.duplicated().sum()

0

Исправим одинаковые строки написаные в разных регистрах в столбце education

In [24]:
data['education'].value_counts()

среднее                13564
высшее                  4658
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   269
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [25]:
def lowercase_education(row):
    education = row['education']
    return education.lower()


data['education'] = data.apply(lowercase_education, axis=1)

In [26]:
data['education'].value_counts()

среднее                15028
высшее                  5193
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

In [27]:
data['family_status'].value_counts()

женат / замужем          12218
гражданский брак          4125
Не женат / не замужем     2780
в разводе                 1179
вдовец / вдова             946
Name: family_status, dtype: int64

In [28]:
def lowercase_family_status(row):
    family_status = row['family_status']
    return family_status.lower()


data['family_status'] = data.apply(lowercase_family_status, axis=1)

In [29]:
data['family_status'].value_counts()

женат / замужем          12218
гражданский брак          4125
не женат / не замужем     2780
в разводе                 1179
вдовец / вдова             946
Name: family_status, dtype: int64

In [30]:
data['income_type'].value_counts()

сотрудник          10968
компаньон           5029
пенсионер           3800
госслужащий         1445
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

**Вывод**

Мы удалили маловероятные полные совпадения строк в таблице, а также заменили повторы набранные в разном регистре в столбцах education и family_status. В других столбцах повторов не обнаружено.

### Лемматизация

In [31]:
data['purpose'].value_counts()

свадьба                                   787
на проведение свадьбы                     764
сыграть свадьбу                           759
операции с недвижимостью                  669
покупка коммерческой недвижимости         656
покупка жилья для сдачи                   648
операции с коммерческой недвижимостью     643
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
покупка своего жилья                      618
строительство жилой недвижимости          618
покупка недвижимости                      613
ремонт жилью                              602
покупка жилой недвижимости                599
на покупку своего автомобиля              501
заняться высшим образованием      

Список целей кредита довольно разнообразный. Для удобства будет полезно сгруппировать цели в несколько общих целей. Мы можем видеть 3 крупных группы: приобретение/строительство/ремонт недвижимости, покупка автомобиля, образование. Среди целей с недвижимостью можно попробовать выделить отдельно покупку коммерческой недвижимости, покупку/строительство жилья. Детализировать группы по автомобилям и образованию уверенно однозначно не удаётся, но всё же попробуем выделить подержанные автомобили и высшее образование. Остальные кредиты (такие как "на свадьбу", "ремонт") отнесём к группе "прочее".

Применим библиотеку nltk для выделения лемм в описании цели, и затем по ключевым и дополнительным лемма разобъём все цели на группы:

Итак, сформируем следующие группы: 
1. покупка/строительство недвижимости (леммы: жил, недвижим (кроме 'ремонт') )
2. покупка коммерческой недвижимости (леммы: жил, недвижим, коммерческ) 
3. покупка автомобиля (леммы: автомобил, автомоб) 
4. покупка подержанного автомобиля (леммы: автомобил, автомоб, подержа) 
5. образование (леммы: образован) 
6. высшее образование (леммы: образован, высш) 
7. прочее

In [32]:
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')

purpose_list = data['purpose'].unique()

In [33]:
# функция лемматизации целей получения кредита
def lemmatize_purpose(text):
    purpose = text
    purpose_stemmed = []
    for word in purpose.split(' '):
        stemmed_word = russian_stemmer.stem(word)
        purpose_stemmed.append(stemmed_word)
 
    if ('жил' in purpose_stemmed) or ('недвижим' in purpose_stemmed):
        if ('коммерческ' in purpose_stemmed) or ('сдач' in purpose_stemmed):
            return 'покупка коммерческой недвижимости'
        elif (not 'ремонт' in purpose_stemmed):
            return 'покупка/строительство недвижимости'
    if ('автомобил' in purpose_stemmed) or ('автомоб' in purpose_stemmed):
        if ('подержа' in purpose_stemmed):
            return 'покупка подержанного автомобиля'
        else:
            return 'покупка автомобиля'
    if ('образован' in purpose_stemmed):
        if ('высш' in purpose_stemmed):
            return 'высшее образование'
        else:
            return 'образование'
    else:
        return 'прочее'
    
def lemmatize_purpose_row(row):
    text = row['purpose']
    return lemmatize_purpose(text)

for item in purpose_list:
    print(lemmatize_purpose(item), '-', item)

покупка/строительство недвижимости - покупка жилья
покупка автомобиля - приобретение автомобиля
образование - дополнительное образование
прочее - сыграть свадьбу
покупка/строительство недвижимости - операции с жильем
образование - образование
прочее - на проведение свадьбы
покупка/строительство недвижимости - покупка жилья для семьи
покупка/строительство недвижимости - покупка недвижимости
покупка коммерческой недвижимости - покупка коммерческой недвижимости
покупка/строительство недвижимости - покупка жилой недвижимости
покупка/строительство недвижимости - строительство собственной недвижимости
покупка/строительство недвижимости - недвижимость
покупка/строительство недвижимости - строительство недвижимости
покупка подержанного автомобиля - на покупку подержанного автомобиля
покупка автомобиля - на покупку своего автомобиля
покупка коммерческой недвижимости - операции с коммерческой недвижимостью
покупка/строительство недвижимости - строительство жилой недвижимости
покупка/строительств

Cоздадим столбец с укрупнёнными целями кредита 

In [34]:
# создадим столбец с укрупнёнными целями кредита 
data['purpose_type'] = data.apply(lemmatize_purpose_row, axis=1)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,покупка/строительство недвижимости
1,1,4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,покупка автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,покупка/строительство недвижимости
3,3,4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,прочее


**Вывод**

Цели кредита были разбиты на 7 типов: 3 крупные связанные (на недвижимость, автомобиль и образование) и все остальные. Из недвижимости выделена группа на покупку коммерческой недвижимости, в автомобилях выделена группа покупки подержанных автомобилей, в образовании - высшее образование.

### Категоризация данных

Исследование требует рассмотреть зависимости по ежемесячному доходу. Возьмём квази логарифмическую шкалу доходов для наглядности, где среднее значение дохода из данных попадает в середину среднего уровня, а симметричные уровни имеют приблизительно одинаковое количество клиентов:
    до 50 тыс. - очень низкий уровень
    50 - 100 тыс. низкий уровень
    100 - 250 тыс. средний уровень
    250 - 500 тыс. высокий уровень
    свыше 500 тыс. очень высокий уровень  

In [35]:
# функция сопоставления дохода группе
def total_income_group(total_income):
    if total_income < 50000:
        return 'до 50 тыс.'
    if 50000 <= total_income < 100000:
        return '50 - 100 тыс.'
    if 100000 <= total_income < 250000:
        return '100 - 250 тыс.'
    if 250000 <= total_income < 500000:
        return '250 - 500 тыс.'
    if 500000 <= total_income:
        return 'свыше 500 тыс.'


In [36]:
data['total_income_group'] = data['total_income'].apply(total_income_group)
data.groupby('total_income_group').sum()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
total_income_group,,,,,,,
100 - 250 тыс.,6681,708720151,604521,11470,13831,1178,2252818636
250 - 500 тыс.,1283,100389913,109558,1690,2428,179,813291936
50 - 100 тыс.,1807,406477254,182026,3754,3962,330,322129705
до 50 тыс.,160,65653907,18401,371,263,23,15462830
свыше 500 тыс.,133,7485876,9560,91,190,14,155053912


Мы имеем возможность оценить сгруппировать клиенов банка по возрасту, разделим на 4 группы:
    до 25 лет (юные, активно обучающаяся)
    25 - 45 (молодёжь)
    45 - 65 (средний возраст)
    старше 65 (пожилые).

In [37]:
# функция сопоставления возраста возрастной группе
def age_group(age):
    if age < 25:
        return 'до 25 лет'
    if 25 <= age < 45:
        return '25 - 45 лет'
    if 45 <= age < 65:
        return '45 - 65 лет'
    if 65 <= age:
        return 'старше 65 лет'

In [38]:
data['dob_years_group'] = data['dob_years'].apply(age_group)
data.groupby('dob_years_group').sum()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
dob_years_group,,,,,,,
25 - 45 лет,8129,49286485,376101,8355,10086,1032,1881092874
45 - 65 лет,1642,981840678,467841,7358,7908,555,1428051081
до 25 лет,254,1075659,19707,827,1712,88,126337606
старше 65 лет,39,256524279,60417,836,968,49,123275458


## Ответы на вопросы

Рассчитаем средний процент "должников".

In [39]:
"{:,.1%}".format((data['debt'].sum() / data['debt'].count()))

'8.1%'

Будем ориентироваться на среднее количество должников 8%

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Рассчитаем процент клиентов, которые задерживали выплаты по кредиту, поделим количество кредитов с задержками на общее количество и расчитаем процент по категориям.

In [40]:
children_pivot = data.pivot_table(index=['children'], values='debt',
                                  aggfunc={
                                    'debt': [len,  sum]
                                  })
children_pivot['debt_rate'] = (children_pivot['sum']/children_pivot['len'])
children_pivot['debt_rate_percent'] = (children_pivot['sum']/children_pivot['len']).map('{:,.1%}'.format)
children_pivot.sort_values(by='debt_rate',  ascending=False)

,len,sum,debt_rate,debt_rate_percent
children,,,,
4,41,4,0.097561,9.8%
2,2039,194,0.095145,9.5%
1,4793,441,0.092009,9.2%
3,328,27,0.082317,8.2%
0,14038,1058,0.075367,7.5%
5,9,0,0.000000,0.0%


**Вывод**

Зависимость между наличием детей и возвратом кредита в срок незначительная. Процент должников близок к среднему. 

Процент задолжников в каждой группе приблизительно одинаковый (7-10%). В лушую сторону отличаются бездетные. В худшую - семьи с 2-мя и 4-мя детьми Исключением здесь являются многодетные семьи с 5-ю детьми, которые всегда выплачивали кредиты вовремя. Но, так как выборка для этой группы очень мала, можно не рассматривать этот процент в итоговом выводе.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Рассчитаем процент клиентов, которые задерживали выплаты по кредиту, поделим количество кредитов с задержками на общее количество и расчитаем процент по категориям.

In [88]:
family_status_pivot = data.pivot_table(index=['family_status'], values='debt',
                                  aggfunc={
                                    'debt': [len,  sum]
                                  })
family_status_pivot['debt_rate'] = (family_status_pivot['sum']/family_status_pivot['len'])
family_status_pivot['debt_rate_percent'] = (family_status_pivot['sum']/family_status_pivot['len']).map('{:,.1%}'.format)
family_status_pivot.sort_values(by='debt_rate',  ascending=False)

,len,sum,debt_rate,debt_rate_percent
family_status,,,,
не женат / не замужем,2780,272,0.097842,9.8%
гражданский брак,4113,383,0.093119,9.3%
женат / замужем,12213,923,0.075575,7.6%
в разводе,1179,84,0.071247,7.1%
вдовец / вдова,946,62,0.065539,6.6%


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок незначительная. Процент должников близок к среднему. 

Процент задолжников в каждой группе приблизительно одинаковый (7-10%). Причём люди не состоящие в браке оказались более беспечными в долговых обязательствах. Банкам можно чуть больше доверять вдовцам и разведённым.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Рассчитаем процент клиентов, которые задерживали выплаты по кредиту, поделим количество кредитов с задержками на общее количество и расчитаем процент по категориям.

In [89]:
total_income_group_pivot = data.pivot_table(index=['total_income_group'], values='debt',
                                  aggfunc={
                                    'debt': [len,  sum]
                                  })
total_income_group_pivot['debt_rate'] = (total_income_group_pivot['sum']/total_income_group_pivot['len'])
total_income_group_pivot['debt_rate_percent'] = (total_income_group_pivot['sum']/total_income_group_pivot['len']).map('{:,.1%}'.format)
total_income_group_pivot.sort_values(by='debt_rate',  ascending=False)

,len,sum,debt_rate,debt_rate_percent
total_income_group,,,,
100 - 250 тыс.,14027,1178,0.083981,8.4%
50 - 100 тыс.,4052,330,0.081441,8.1%
250 - 500 тыс.,2561,179,0.069895,7.0%
свыше 500 тыс.,222,14,0.063063,6.3%
до 50 тыс.,369,23,0.062331,6.2%


**Вывод**

Нет. Зависимость между ежемесячным доходом и возвратом кредита в срок незначительная. Процент должников близок к среднему. 

Процент задолжников в каждой группе приблизительно одинаковый (6-8%). Клиенты с самыми низким доходом и с очень высоким оказались чуть более дисциплинированными.

### Как разные цели кредита влияют на его возврат в срок?

Рассчитаем процент клиентов, которые задерживали выплаты по кредиту, поделим количество кредитов с задержками на общее количество и расчитаем процент по категориям.

In [90]:
purpose_type_pivot = data.pivot_table(index=['purpose_type'], values='debt',
                                  aggfunc={
                                    'debt': [len,  sum]
                                  })
purpose_type_pivot['debt_rate'] = (purpose_type_pivot['sum']/purpose_type_pivot['len'])
purpose_type_pivot['debt_rate_percent'] = (purpose_type_pivot['sum']/purpose_type_pivot['len']).map('{:,.1%}'.format)
purpose_type_pivot.sort_values(by='debt_rate',  ascending=False)

,len,sum,debt_rate,debt_rate_percent
purpose_type,,,,
высшее образование,1359,129,0.094923,9.5%
покупка автомобиля,3308,311,0.094015,9.4%
образование,2611,240,0.091919,9.2%
покупка подержанного автомобиля,950,86,0.090526,9.1%
покупка коммерческой недвижимости,1945,150,0.077121,7.7%
прочее,2901,216,0.074457,7.4%
покупка/строительство недвижимости,8157,592,0.072576,7.3%


**Вывод**

Цели кредита влияют на возврат кредита в срок не значительно.

В этой группировке более рискованные цели оказались образование и покупка автомобиля, в них процент должников выше среднего. В то время как среди тех, кто брал кредит на недвижимость и свадьбы, клиенты выполняли свои обязательства лучше.

Проверим разброс должников по возрастам.

Рассчитаем процент клиентов, которые задерживали выплаты по кредиту, поделим количество кредитов с задержками на общее количество и расчитаем процент по категориям.

In [91]:
dob_years_group_pivot = data.pivot_table(index=['dob_years_group'], values='debt',
                                  aggfunc={
                                    'debt': [len,  sum]
                                  })
dob_years_group_pivot['debt_rate'] = (dob_years_group_pivot['sum']/dob_years_group_pivot['len'])
dob_years_group_pivot['debt_rate_percent'] = (dob_years_group_pivot['sum']/dob_years_group_pivot['len']).map('{:,.1%}'.format)
dob_years_group_pivot.sort_values(by='debt_rate',  ascending=False)

,len,sum,debt_rate,debt_rate_percent
dob_years_group,,,,
до 25 лет,871,88,0.101033,10.1%
25 - 45 лет,10763,1032,0.095884,9.6%
45 - 65 лет,8704,555,0.063764,6.4%
старше 65 лет,893,49,0.054871,5.5%


Здесь мы можем увидеть заметную разницу. Люди старшего возраста почти в 2 раза реже задерживают выплату по сравнению с молодёжью.

## Общий вывод

В целом сравнения показали, что зависимость по перечисленным характеристикам не значительная, процент  должников колеблется в пределах 3 единиц. За ориентир был взят средний процент проблемных кредитов - 8,1%.

Есть ли зависимость между наличием детей и возвратом кредита в срок?
    Бездетные в среднем лучше возвращают кредит, чем имеющие детей.
    
Есть ли зависимость между семейным положением и возвратом кредита в срок?
    Клиенты не состоящие в браке, или живущие в гражданском браке в среднем хуже возвращают долги.
    
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    Клиенты с доходами более 250 тыс., а также с доходами менее 50 тыс. относятся к возврату кредитов аккуратнее.
    
Как разные цели кредита влияют на его возврат в срок? 
    Кредиты на образование и автокредиты чаще возвращают с задержками.
    

Дополнительное детализирование целей кредитования не принесло интересных результатов. Кредиты на образование и на высшее образование имеют одинаковые показатели, также в недвижимости и автомобилях.


Дополнительная выборка должников по возрастам, показала, что люди старшего в среднем чаще возвращают кредиты в срок. Возможно, для более контрастных результатов следует провести более детальное исследование с использованием возрастных групп.